In [ ]:
basedir = '/home/jun/run/'

In [ ]:
import os
from typing import Dict, List, Tuple
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint
from matplotlib.lines import Line2D


def find_top5_asymmetry(data: np.ndarray) -> Tuple:
    asymmetry_matrix = np.abs(np.triu(data, k=1) - np.tril(data, k=-1).T)
    asymmetry_combinations = np.triu(asymmetry_matrix, k=1)
    top5_indices = np.unravel_index(np.argsort(
        asymmetry_combinations, axis=None)[-5:], asymmetry_combinations.shape)
    return top5_indices


def draw_heatmap(title: str, data: np.ndarray, eventList: list) -> None:
    xticklabels = eventList
    yticklabels = list(reversed(eventList))
    matrix_flipped = np.flipud(data)
    ax = sns.heatmap(matrix_flipped, xticklabels=xticklabels,
                     yticklabels=yticklabels)
    plt.gcf().set_size_inches(10, 8)
    line = Line2D([matrix_flipped.shape[1], 0], [
        0, matrix_flipped.shape[0]], color='red')
    plt.gca().add_line(line)
    plt.title(title)
    plt.show()


def dump_heatmap(title: str, data: np.ndarray, eventList: list, directory: str) -> None:
    xticklabels = eventList
    yticklabels = list(reversed(eventList))
    matrix_flipped = np.flipud(data)
    ax = sns.heatmap(matrix_flipped, xticklabels=xticklabels,
                     yticklabels=yticklabels)

    # plt.gcf().set_size_inches(10, 8)

    line = Line2D([matrix_flipped.shape[1], 0], [
        0, matrix_flipped.shape[0]], color='red')
    plt.gca().add_line(line)
    plt.title(title)
    os.chdir(directory)
    # plt.savefig(title+".png", box_inches=)


def do_normalization(bench: str, matrix: np.ndarray, eventList: list) -> np.ndarray:
    path = os.path.join(basedir, bench, "m5out")
    os.chdir(path)
    numbers = []
    event_counts = {}
    i = int()
    with open('stats.txt', 'r') as f:
        lines = f.readlines()
    for line in lines:
        if "simInsts" in line:
            i = int(line.split()[1])/1000
    for event in eventList:
        count = sum(event in line for line in lines)
        event_counts[event] = count
    for event in eventList:
        for line in lines:
            if event in line and ('total' in line or event_counts[event] == 1):
                numbers.append(int(line.split()[1])/i)
                break
    return matrix / np.array(numbers)

In [ ]:
import os
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint
from matplotlib.lines import Line2D


def analyze(bench: str, normalize: bool = False) -> Tuple:
    map = {}
    top_pairs = []

    path = os.path.join(basedir, bench)
    os.chdir(path)

    with open(bench+'.out', 'r') as f:
        lines = []
        text = ""
        for line in f:
            if line.startswith('recorded events:'):
                eventList = line.split(":")[1].strip().split(" ")
            lines.append(line)
        text = "".join(lines)

    match1 = re.search(r"Executed instructions: (\d+)", text)
    if match1:
        executed_instructions = int(match1.group(1))
        i = executed_instructions / 1000
    else:
        print("No executed_instructions found.")

    # print(bench + ": ", executed_instructions)

    match3 = re.search(r"all events count: (\d+)", text)
    if match3:
        event_count = int(match3.group(1))
    else:
        print("No event_count found.")

    assert len(eventList) == event_count, "event count mismatch"
    map[bench] = event_count
    del event_count

    fd = 'bc16v1.txt'
    with open(fd, 'r') as file:
        data = file.read()
    data_list = eval(data)
    m16v1 = np.array(data_list, np.uint64) / i

    fd = 'bc16v2.txt'
    with open(fd, 'r') as file:
        data = file.read()
    data_list = eval(data)
    m16v2 = np.array(data_list, np.uint64) / i

    fd = 'bc32v1.txt'
    with open(fd, 'r') as file:
        data = file.read()
    data_list = eval(data)
    m32v1 = np.array(data_list, np.uint64) / i

    fd = 'bc32v2.txt'
    with open(fd, 'r') as file:
        data = file.read()
    data_list = eval(data)
    m32v2 = np.array(data_list, np.uint64) / i

    fd = 'bc64v1.txt'
    with open(fd, 'r') as file:
        data = file.read()
    data_list = eval(data)
    m64v1 = np.array(data_list, np.uint64) / i

    fd = 'bc64v2.txt'
    with open(fd, 'r') as file:
        data = file.read()
    data_list = eval(data)
    m64v2 = np.array(data_list, np.uint64) / i

    fd = 'bc128v1.txt'
    with open(fd, 'r') as file:
        data = file.read()
    data_list = eval(data)
    m128v1 = np.array(data_list, np.uint64) / i

    fd = 'bc128v2.txt'
    with open(fd, 'r') as file:
        data = file.read()
    data_list = eval(data)
    m128v2 = np.array(data_list, np.uint64) / i

    fd = 'bc256v1.txt'
    with open(fd, 'r') as file:
        data = file.read()
    data_list = eval(data)
    m256v1 = np.array(data_list, np.uint64) / i

    fd = 'bc256v2.txt'
    with open(fd, 'r') as file:
        data = file.read()
    data_list = eval(data)
    m256v2 = np.array(data_list, np.uint64) / i

    m32v1 = m32v1 + m16v1
    m32v2 = m32v2 + m16v2
    m64v1 = m64v1 + m32v1
    m64v2 = m64v2 + m32v2
    m128v1 = m128v1 + m64v1
    m128v2 = m128v2 + m64v2
    m256v1 = m256v1 + m128v1
    m256v2 = m256v2 + m128v2

    if normalize == True:
        for i, m in enumerate([m16v1, m16v2, m32v1, m32v2, m64v1, m64v2, m128v1, m128v2, m256v1, m256v2]):
            m = do_normalization(bench, m, eventList)
            if i == 0:
                m16v1 = m
            elif i == 1:
                m16v2 = m
            elif i == 2:
                m32v1 = m
            elif i == 3:
                m32v2 = m
            elif i == 4:
                m64v1 = m
            elif i == 5:
                m64v2 = m
            elif i == 6:
                m128v1 = m
            elif i == 7:
                m128v2 = m
            elif i == 8:
                m256v1 = m
            elif i == 9:
                m256v2 = m

    top5_asymmetry_results = {}
    top5_asymmetry_results[bench] = find_top5_asymmetry(m256v2)
    # indices in index of event
    indices = top5_asymmetry_results[bench]
    top_pairs = [(eventList[i], eventList[j]) for i, j in zip(*indices)]

    max_index = np.unravel_index(np.argmax(m16v2, axis=None), m16v2.shape)
    top16 = (eventList[max_index[0]], eventList[max_index[1]])

    max_index = np.unravel_index(np.argmax(m32v2, axis=None), m32v2.shape)
    top32 = (eventList[max_index[0]], eventList[max_index[1]])

    max_index = np.unravel_index(np.argmax(m64v2, axis=None), m64v2.shape)
    top64 = (eventList[max_index[0]], eventList[max_index[1]])

    max_index = np.unravel_index(np.argmax(m128v2, axis=None), m128v2.shape)
    top128 = (eventList[max_index[0]], eventList[max_index[1]])

    max_index = np.unravel_index(np.argmax(m256v2, axis=None), m256v2.shape)
    top256 = (eventList[max_index[0]], eventList[max_index[1]])

    tops = zip("top16, top32, top64, top128, top256".split(", "),
               [top16, top32, top64, top128, top256])

    m = {key: value for (key, value) in zip("m16v2, m32v2, m64v2, m128v2, m256v2".split(
        ", "), [m16v2, m32v2, m64v2, m128v2, m256v2])}

    # return matrixs[bench], top_pairs[bench], tops[bench], events[bench]
    return m, top_pairs, dict(tops), eventList

In [15]:
benchlist = '''ahamont64 crc32 cubic edn huffbench matmultint md5sum minver nbody nettleaes
    nettlesha256 nsichneu picojpeg primecount qrduino slre st statemate tarfind ud wikisort'''.split()
# benchlist = '''crc32'''.split()
tops = {}
matrixs = {}
top_pairs = {}
events = {}

for bench in benchlist:
    matrixs[bench], top_pairs[bench], tops[bench], events[bench] = analyze(
        bench)

In [14]:
benchlist = '''ahamont64 crc32 cubic edn huffbench matmultint md5sum minver nbody nettleaes
    nettlesha256 nsichneu picojpeg primecount qrduino slre st statemate tarfind ud wikisort'''.split()
# benchlist = '''crc32'''.split()
tops_n = {}
matrixs_n = {}
top_pairs_n = {}
events = {}

for bench in benchlist:
    matrixs_n[bench], top_pairs_n[bench], tops_n[bench], events[bench] = analyze(
        bench, normalize=True)

In [ ]:
# for bench in benchlist:
#     draw_heatmap(bench+", 16 cycle", matrixs[bench]['m16v2'], events[bench])
#     draw_heatmap(bench+", 256 cycle", matrixs[bench]['m256v2'], events[bench])

In [ ]:
for bench in benchlist:
    draw_heatmap(bench+", normalized, 16 cycle",
                 matrixs_n[bench]['m16v2'], events[bench])
    draw_heatmap(bench+", normalized, 256 cycle",
                 matrixs_n[bench]['m256v2'], events[bench])

In [ ]:
for bench in benchlist:
    print(bench)
    pprint(tops[bench])
    print("top 5 pairs")
    pprint(top_pairs[bench])
    # newline
    print()

In [17]:
for bench in benchlist:
    print(bench)
    pprint(tops_n[bench])
    print("top 5 pairs")
    pprint(top_pairs_n[bench])
    # newline
    print()

ahamont64
{'top128': ('rename.idleCycles', 'l2cache.overallHits'),
 'top16': ('rename.idleCycles', 'l2cache.overallHits'),
 'top256': ('rename.idleCycles', 'l2cache.overallHits'),
 'top32': ('l1dcache.overallMisses', 'l2cache.overallHits'),
 'top64': ('rename.idleCycles', 'l2cache.overallHits')}
top 5 pairs
[('l2cache.overallHits', 'l2cache.overallMisses'),
 ('l1dcache.overallMisses', 'l2cache.overallHits'),
 ('decode.branchResolved', 'l2cache.overallHits'),
 ('decode.idleCycles', 'l2cache.overallHits'),
 ('l2cache.overallHits', 'rename.idleCycles')]

crc32
{'top128': ('decode.idleCycles', 'l2cache.overallHits'),
 'top16': ('branchPred.condIncorrect', 'branchPred.RASIncorrect'),
 'top256': ('rename.idleCycles', 'l2cache.overallHits'),
 'top32': ('l1dcache.overallMisses', 'l2cache.overallHits'),
 'top64': ('rename.idleCycles', 'l2cache.overallHits')}
top 5 pairs
[('l2cache.overallHits', 'l2cache.overallMisses'),
 ('l1dcache.overallMisses', 'l2cache.overallHits'),
 ('decode.branchResolve

In [ ]:
count = dict()
for bench in benchlist:
    for pair in top_pairs[bench]:
        count[pair] = count.get(pair, 0) + 1

print("Top pairs:")
pprint(count)

In [16]:
count = dict()
for bench in benchlist:
    for pair in top_pairs_n[bench]:
        count[pair] = count.get(pair, 0) + 1

print("Top pairs:")
pprint(count)

Top pairs:
{('branchPred.RASIncorrect', 'branchPred.RASUsed'): 1,
 ('branchPred.RASIncorrect', 'decode.branchResolved'): 2,
 ('branchPred.RASIncorrect', 'iew.predictedTakenIncorrect'): 1,
 ('decode.branchResolved', 'l2cache.overallHits'): 21,
 ('decode.idleCycles', 'l2cache.overallHits'): 21,
 ('l1dcache.overallMisses', 'l2cache.overallHits'): 21,
 ('l2cache.overallHits', 'l2cache.overallMisses'): 17,
 ('l2cache.overallHits', 'rename.idleCycles'): 21}
